In [0]:
# Install Pyomo 
%pip install pyomo --quiet
dbutils.library.restartPython()

In [0]:
import os
import mlflow
import numpy as np
import pandas as pd
import pyomo.environ as pyo

In [0]:
%sh bash install_cbc.sh

In [0]:
# ---------------------------------------------------------------
# Builds a synthetic 2-tier network dataset for optimisation.
# ---------------------------------------------------------------

import random, string, math
import matplotlib.pyplot as plt
from itertools import product
from collections import defaultdict

# ------------- 0 — reproduce the exact network ------------------
N1, N2 = 10, 20
random.seed(777)                 # <- keep topology reproducible

tier1 = [f"T1_{i}" for i in range(1, N1 + 1)]
tier2 = [f"T2_{i}" for i in range(1, N2 + 1)]

edges = []                         # (src, tgt)

# ---- Tier-2 → Tier-1  (1–3 edges out of each Tier-2) ----------
t2_out = {t2: set() for t2 in tier2}
shuffled_t2 = tier2.copy()
random.shuffle(shuffled_t2)
for t1_node, t2_node in zip(tier1, shuffled_t2):
    edges.append((t2_node, t1_node))
    t2_out[t2_node].add(t1_node)

for t2_node in tier2:
    desired = random.randint(1, 3)
    while len(t2_out[t2_node]) < desired:
        candidate = random.choice(tier1)
        if candidate not in t2_out[t2_node]:
            edges.append((t2_node, candidate))
            t2_out[t2_node].add(candidate)

# ------------- 1 — scalar & tabular parameters ------------------
rng_int   = lambda lo, hi: random.randint(lo, hi)
rng_float = lambda lo, hi, r=2: round(random.uniform(lo, hi), r)

# Profit margin for finished products
f = {j: rng_float(0.05, 0.30) for j in tier1}

# On-hand inventory for product nodes
s = {j: rng_int(400, 1800) for j in tier1}

# Demand per TTR for finished products
d = {j: rng_int(400, 1800) for j in tier1}

# Production capacity per TTR for the supplier nodes
c = {a: rng_int(500, 2500) for a in tier2}

# Time-to-recover for this disruption scenario
t = 1

# A small share of Tier-2 + Tier-3 nodes disrupted
disrupted_count = max(1, int(0.10 * (len(tier2))))
disrupted       = random.sample(tier2, disrupted_count)

# ------------- 2 — quick smoke test when run directly ----------
if __name__ == "__main__":
    print("Tier sizes  :", len(tier1), len(tier2))
    print("Edges       :", len(edges))
    print("Disrupted   :", disrupted)
    print("------------------------------------------------------")
    print("f:", {j: f[j] for j in tier1})
    print("d:", {j: d[j] for j in tier1})
    print("s:", {j: s[j] for j in tier1})
    print("c:", {j: c[j] for j in tier2})
    print(f"edges:", edges)

In [0]:
# -------------------------------------------------------------
# COLOUR MAP  (one colour per distinct profit margin)
# -------------------------------------------------------------

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

profit_codes = sorted(set(f.values()))              # [0.10, 0.15, 0.20]
cmap          = plt.get_cmap("YlGnBu", len(profit_codes))
code_colour   = {p: cmap(i) for i, p in enumerate(profit_codes)}

default_colour = "#9e9e9e"      # fallback for A-nodes

def colours_for(nodes):
    """Return list of node colours in the given order."""
    return [code_colour.get(f.get(n), default_colour) for n in nodes]

# -------------------------------------------------------------
# POSITIONS  (two centred tiers)
# -------------------------------------------------------------
pos = {}

gap_t1 = 2.5
gap_t2 = 2.5

tier_specs = [                 # (nodes, gap, y-coordinate)
    (tier1, gap_t1, 1),             # Tier-1: products
    (tier2, gap_t2, 0),             # Tier-2: vendors
]

max_width = max((len(nodes) - 1) * gap for nodes, gap, _ in tier_specs)

for nodes, gap, y in tier_specs:
    width    = (len(nodes) - 1) * gap
    x_offset = (max_width - width) / 2          # centre the tier
    for idx, node in enumerate(nodes):
        pos[node] = (x_offset + idx * gap, y)

# -------------------------------------------------------------
# VISUALISATION
# -------------------------------------------------------------
fig, ax = plt.subplots(figsize=(13, 5))

# Tier-1  (Product-nodes, grey circles)
ax.scatter([pos[n][0] for n in tier1], [pos[n][1] for n in tier1],
           s=600, marker='o', c=colours_for(tier1),
           edgecolor='k', linewidth=0.6, label="Tier 1 (products)")

# Tier-2  (Supplier-nodes, coloured squares)
ax.scatter([pos[n][0] for n in tier2], [pos[n][1] for n in tier2],
           s=600, marker='s', c=colours_for(tier2),
           edgecolor='k', linewidth=0.6, label="Tier 2 (suppliers)")

# Node labels
for n, (x, y) in pos.items():
    ax.text(x, y, n, ha='center', va='center', fontsize=8)

# Directed edges
for src, tgt in edges:
    sx, sy = pos[src]
    tx, ty = pos[tgt]
    ax.annotate("",
                xy=(tx, ty), xytext=(sx, sy),
                arrowprops=dict(arrowstyle="-|>", lw=0.8))

# Axes & title
ax.set_xlim(-2.5, max_width + 2.5)
ax.set_ylim(-0.5, 1.5)
ax.axis("off")
plt.title("Two-Tier Directed Network\n(coloured by profit margin)")

# Custom legend: one patch per profit margin value
patches = [mpatches.Patch(color=code_colour[p], label=f"π = {p:.2f}")
           for p in profit_codes]
first_legend = ax.legend(handles=patches, title="profit margin",
                         fontsize=8, title_fontsize=9,
                         loc="upper left", bbox_to_anchor=(1.02, 1))
ax.legend(loc="upper left", bbox_to_anchor=(1.02, 0.6))
ax.add_artist(first_legend)     # keep both legends

plt.tight_layout()
plt.show()

In [0]:
# optimisation_model.py
from pyomo.environ import *

# ------------------------------------------------------------------
# 1.  Prepare your data
# ------------------------------------------------------------------
data = {
    # elementary sets ------------------------------------------------
    'V'      : tier1,               # product nodes
    'A'      : tier2,               # all BUT leaf nodes
    'E'      : edges,               # all edges
    'S'      : disrupted,           # disrupted nodes in scenario n
    # parameters -----------------------------------------------------
    'f'  : f,    # profit margin of 1 unit of j
    's'  : s,    # finished-goods inventory of j
    't'  : t,    # TTR for disruption scenario n   (a scalar)
    'd'  : d,    # demand for j per TTR
    'c'  : c,    # plant capacity per TTR
}


# ------------------------------------------------------------------
# 2.  Build the ConcreteModel
# ------------------------------------------------------------------
m = ConcreteModel()

# ---- 2.1  sets ----------------------------------------------------
m.V   = Set(initialize=data['V'])
m.A   = Set(initialize=data['A'])
m.S   = Set(initialize=data['S'])
m.E   = Set(initialize=data['E'])

# handy union of *all* nodes that may carry production volume
m.NODES = pyo.Set(initialize=list(set(data['V']) | set(data['A'])))

# ---- 2.2  parameters ---------------------------------------------
m.f = Param(m.V, initialize=data['f'], within=NonNegativeReals)          # impact (profit margin)
m.s = Param(m.V, initialize=data['s'], within=NonNegativeIntegers)
m.t = Param(initialize=data['t'], within=PositiveReals)
m.d = Param(m.V, initialize=data['d'], within=NonNegativeIntegers)
m.c = Param(m.A, initialize=data['c'], within=NonNegativeIntegers)

# ---- 2.3  decision variables -------------------------------------
m.l = Var(m.V, domain=NonNegativeIntegers)          # lost volume of product j
m.y = Var(m.E, domain=NonNegativeIntegers)


# ------------------------------------------------------------------
# 3.  objective
# ------------------------------------------------------------------
def obj_rule(mdl):
    return sum(mdl.f[j] * mdl.l[j] for j in mdl.V)
m.OBJ = Objective(rule=obj_rule, sense=minimize)


# ------------------------------------------------------------------
# 4.  constraints
# ------------------------------------------------------------------
# Σ_{i:(i,j)∈F⁽ⁿ⁾} y_ij + l_j ≥ d_j · t⁽ⁿ⁾ - s_j,            ∀ j∈𝒱
def demand_rule(mdl, j):
    return sum(mdl.y[i,j] for (i,j) in mdl.E) + mdl.l[j] >= mdl.d[j] * mdl.t - mdl.s[j]
m.Demand = Constraint(m.V, rule=demand_rule)

# Σ_{k∈𝒜_α} u_k ≤ c_α · t⁽ⁿ⁾,                ∀ j∈NODES
def capacity_rule(mdl, i):
    return sum(mdl.y[i, j] for (i,j) in m.E) <= m.c[i] * mdl.t
m.Capacity = Constraint(m.A, rule=capacity_rule)


# ------------------------------------------------------------------
# 5.  solve
# ------------------------------------------------------------------
if __name__ == "__main__":
    # choose any LP/MIP solver that Pyomo can see (CBC, Gurobi, CPLEX, HiGHS, …)
    solver = SolverFactory("cbc")      # just an example
    result = solver.solve(m, tee=True)
    m.display()                        # quick sanity-check of results
